### Первая лаба - написать код для решения системы линейных уравнений методом Гаусса и методом Гаусса с выбором главного элемента.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as smp
import scipy as sp

Система вида

$$
\begin{cases}
    a_{11}x_1+a_{12}x_2+\dots+a_{1n}x_n=b_1
    \\
    a_{21}x_1+a_{22}x_2+\dots+a_{2n}x_n=b_2
    \\
    \dots
    \\
    a_{n1}x_1+a_{n2}x_2+\dots+a_{nn}x_n=b_n
\end{cases}
$$

В матричном виде это уравнение

$$
A \cdot \vec{x} = \vec{b}
$$

Матрица этой системы

$$
A = 
\begin{bmatrix}
    a_{11} & a_{12} & \dots & a_{1n}
    \\
    a_{21} & a_{22} & \dots & a_{2n}
    \\
    \vdots & \vdots & \ddots & \vdots
    \\
    a_{n1} & a_{n2} & \dots & a_{nn}
\end{bmatrix}, \space \space
\vec{x} = 
\begin{bmatrix}
    x_1
    \\
    x_2
    \\
    \vdots
    \\
    x_n
\end{bmatrix}, \space \space
\vec{b} = 
\begin{bmatrix}
    b_1
    \\
    b_2
    \\
    \vdots
    \\
    b_n
\end{bmatrix} 
$$

Решение методом Гаусса предполагает приведение матрицы к треугольному виду

$$
\hat{A} = 
\begin{bmatrix}
    1 & c_{12} & c_{13} & \dots & c_{1n}
    \\
    0 & 1 & c_{23} & \dots & c_{2n}
    \\
    \vdots & \vdots & \ddots & \vdots
    \\
    0 & 0 & 0 & \dots & 1
\end{bmatrix}, \space \space
\hat{\vec{b}} = 
\begin{bmatrix}
    \hat{b_1}
    \\
    \hat{b_2}
    \\
    \vdots
    \\
    \hat{b_n}
\end{bmatrix} 
$$
$$

In [47]:
# Исходная система в матричном представлении
A = [
       [ 1.0, -2.0,  3.0,  -4.0],
       [ 3.0,  3.0, -5.0,  1.0],
       [ 3.0,  0.0,  3.0, -10.0],
       [-2.0,  1.0,  2.0,  -3.0]
]
 
B = [
        2.0,
       -3.0,
        8.0,
        5.0
]

In [3]:
# Вывод текущей системы
def print_system(A, B, selected):
    result_string = '\n'
    for row in range(len(B)):
        start = ("/" if row==0 else "\\" if row==len(B)-1 else '|')
        end = ("\\" if row==0 else "/" if row==len(B)-1 else '|')
        result_string += start 
        for col in range(len(A[row])):
            result_string += "{1:7.3f}{0} ".format(" " if (selected is None or selected != (row, col)) else "*", A[row][col]) 
        result_string += end + f" * {start}  x{row+1}  {end} = {start}{B[row]:7.3f}  {end}" + '\n'
    return result_string


class LinearSolver:

    roots = None
    
    # Заполнение матрицы заданными числами
    def fill(self, A, B) -> None:
        self.A = np.array(A, copy=True, dtype=float)
        self.B = np.array(B, copy=True, dtype=float)

    # Рандомное заполнение матрицы
    def random_fill(self, n):
        self.A = np.random.random((n, n))
        self.B = np.random.random((n))

    # Рандомное задание матрицы (с 0 на главной диагонали)
    def random_fill_with_zeros(self, n):
        self.A = np.random.random((n, n))
        self.B = np.random.random((n))
        for i in range(n):
            self.A[i, i] = 0

    # Перестановка строк
    def swap_rows(self, first_row, second_row) -> None:
        self.A[first_row], self.A[second_row] = self.A[second_row], np.copy(self.A[first_row])
        self.B[first_row], self.B[second_row] = self.B[second_row], np.copy(self.B[first_row])

    # Деление строки на число
    def divide_row(self, row, divider) -> None:
        self.A[row] = [a / divider if a != 0 else 0 for a in self.A[row]]
        self.B[row] /= divider

    # Совмещение строк
    def combine_rows(self, to_row, from_row, multiplier) -> None:
        self.A[to_row] = [(a + e * multiplier) for a, e in zip(self.A[to_row], self.A[from_row])]
        self.B[to_row] += self.B[from_row] * multiplier

    # Расчёт ошибки 
    def epsilon(self):
        x  = np.array(self.get_roots(), copy=True)
        x0 = np.linalg.solve(self.A, self.B)
        temp = np.abs(x - x0)
        res = 0
        for e in temp:
            res += e**2
        return np.sqrt(res)
    
    # Расчёт нормы невязки
    def delta(self):
        x  = np.array(self.get_roots(), copy=True)
        return np.linalg.norm(np.abs(self.A @ x - self.B))

    # Получение корней
    def get_roots(self):
        return self.roots

In [4]:
class Gauss(LinearSolver):
    def __init__(self, A=None, B=None):
        if A and B:
            super().__init__(A, B)
            self.roots = [0 for b in range(len(B))]

    # Решение системы методом Гаусса (обычным)
    # с выводом в файл этапов решения
    def solve_(self):
        with open('./src/solved_gauss_.txt', 'w', encoding='utf-8') as file:
            column = 0
            current_step = 0
            lines = []
            while(column < len(self.B)):
                current_step += 1
                line = f"\n{current_step}. "
                line += (f"Ищем ненулевой элемент в {column+1}-м столбце:")
                lines.append(line)
                current_row = None
                for r in range(column, len(self.A)):
                    if (self.A[r][column] != 0):
                        current_row = r
                        break

                if current_row is None:
                    line = ("\nРешений нет. :(")
                    lines.append(line)
                    return None
            
                line = print_system(self.A, self.B, (current_row, column))
                lines.append(line)

                if current_row != column:
                    current_step += 1
                    line = f"\n{current_step}. Переставляем строку с найденным элементом наверх:"
                    lines.append(line)

                    self.swap_rows(current_row, column)

                    line = print_system(self.A, self.B, (column, column))
                    lines.append(line)

                if self.A[column][column] != 1:
                    current_step += 1
                    line = f"\n{current_step}. Нормализуем строку:"
                    if abs(self.A[column][column]) > 1e-12:
                        self.divide_row(column, self.A[column][column])
                        lines.append(line)
                    else:
                        line += ('Мартица несовместна.\nРешений нет. :(')
                        lines.append(line)
                        file.writelines(lines)
                        return None
                    line = print_system(self.A, self.B, (column, column))
                    lines.append(line)

                current_step += 1
                line = (f"\n{current_step}. Обрабатываем строки ниже:")
                lines.append(line)

                for row in range(column+1, len(self.A)):
                    self.combine_rows(row, column, -self.A[row][column])

                line = print_system(self.A, self.B, (column, column))
                lines.append(line)

                column += 1
            
            current_step += 1
            line = (f"\n{current_step}. Матрица приведена к треугольному виду. Обратный ход.\n")
            lines.append(line)
            
            self.roots = [0 for b in self.B]
            for i in range(len(self.B)-1, -1, -1):
                self.roots[i] = self.B[i] - sum(x*a for x, a in zip(self.roots[i+1:], self.A[i][i+1:]))
        
            current_step += 1
            line = (f"\n{current_step}. Ответ:\n")
            line += ("\n".join(f"x{i+1} = {x:7.3f}" for i, x in enumerate(self.roots)))
            lines.append(line)
            
            file.writelines(lines)
        
        return None

    # Решение системы методом Гаусса (улучшенным)
    # с выводом в файл этапов решения
    def solve(self):
        with open('./src/solved_gauss.txt', 'w', encoding='utf-8') as file:
            column = 0
            current_step = 0
            lines = []
            while(column < len(self.B)):
                current_step += 1
                line = f"\n{current_step}. "
                line += (f"Ищем максимальный по модулю элемент в {column+1}-м столбце:")
                lines.append(line)
                current_row = None
                for r in range(column, len(self.A)):
                    if current_row is None or abs(self.A[r][column]) > abs(self.A[current_row][column]):
                        current_row = r

                if current_row is None:
                    line = ("\nРешений нет. :(")
                    lines.append(line)
                    return None
            
                line = print_system(self.A, self.B, (current_row, column))
                lines.append(line)

                if current_row != column:
                    current_step += 1
                    line = f"\n{current_step}. Переставляем строку с найденным элементом наверх:"
                    lines.append(line)

                    self.swap_rows(current_row, column)

                    line = print_system(self.A, self.B, (column, column))
                    lines.append(line)

                if self.A[column][column] != 1:
                    current_step += 1
                    line = f"\n{current_step}. Нормализуем строку:"
                    if abs(self.A[column][column]) > 1e-12:
                        self.divide_row(column, self.A[column][column])
                        lines.append(line)
                    else:
                        line += ('Мартица несовместна.\nРешений нет. :(')
                        lines.append(line)
                        file.writelines(lines)
                        return None
                    line = print_system(self.A, self.B, (column, column))
                    lines.append(line)

                current_step += 1
                line = (f"\n{current_step}. Обрабатываем строки ниже:")
                lines.append(line)

                for row in range(column+1, len(self.A)):
                    self.combine_rows(row, column, -self.A[row][column])

                line = print_system(self.A, self.B, (column, column))
                lines.append(line)

                column += 1
            
            current_step += 1
            line = (f"\n{current_step}. Матрица приведена к треугольному виду. Обратный ход.\n")
            lines.append(line)
            
            self.roots = [0 for b in self.B]
            for i in range(len(self.B)-1, -1, -1):
                self.roots[i] = self.B[i] - sum(x*a for x, a in zip(self.roots[i+1:], self.A[i][i+1:]))
        
            current_step += 1
            line = (f"\n{current_step}. Ответ:\n")
            line += ("\n".join(f"x{i+1} = {x:7.3f}" for i, x in enumerate(self.roots)))
            lines.append(line)
            
            file.writelines(lines)
        
        return None

    # Решение системы методом Гаусса (улучшенным)
    # без вывода в файл этапов решения
    def solve_without_steps(self):
        column = 0
        while(column < len(self.B)):
            current_row = None
            for r in range(column, len(self.A)):
                if current_row is None or abs(self.A[r][column]) > abs(self.A[current_row][column]):
                    current_row = r

            if current_row is None:
                return None
        
            if current_row != column:
                self.swap_rows(current_row, column)

            if A[column][column] != 1:
                if abs(self.A[column][column]) > 1e-12:
                    self.divide_row(column, self.A[column][column])
                else:
                    return None

            for row in range(column+1, len(self.A)):
                self.combine_rows(row, column, -self.A[row][column])
            column += 1
        
        self.roots = [0 for b in self.B]
        for i in range(len(self.B)-1, -1, -1):
            self.roots[i] = self.B[i] - sum(x*a for x, a in zip(self.roots[i+1:], self.A[i][i+1:]))
    
        return None


In [8]:
A = [
    [3, 3, -5],
    [1, 2, -1],
    [2, -1, 4]
]

B = [
    -1, 
    9,
    13
]

In [9]:
print("Исходная система:")
print(print_system(A, B, None))

SolverGauss = Gauss()
SolverGauss.fill(A, B)
SolverGauss.solve()

X = SolverGauss.get_roots()
print('Gauss standart method')
if X is None:
    print('Решений нет. :(')
else:
    print(f'Roots: \t{X}')
    print(f'Error: \t{SolverGauss.epsilon()}')
    print(f'Delta: \t{SolverGauss.delta()}')

print()
    
SolverGauss_ = Gauss()
SolverGauss_.fill(A, B)
SolverGauss_.solve_()

X_ = SolverGauss_.get_roots()
print('Gauss advanced method')
if X_ is None:
    print('Решений нет. :(')
else:
    print(f'Roots: \t{X_}')
    print(f'Error: \t{SolverGauss_.epsilon()}')
    print(f'Delta: \t{SolverGauss_.delta()}')

Исходная система:

/  3.000    3.000   -5.000  \ * /  x1  \ = / -1.000  \
|  1.000    2.000   -1.000  | * |  x2  | = |  9.000  |
\  2.000   -1.000    4.000  / * \  x3  / = \ 13.000  /

Gauss standart method
Roots: 	[0.7500000000000007, 6.357142857142856, 4.4642857142857135]
Error: 	2.220446049250313e-16
Delta: 	1.047382306668854e-15

Gauss advanced method
Roots: 	[0.7499999999999989, 6.357142857142858, 4.4642857142857135]
Error: 	2.220446049250313e-16
Delta: 	5.551115123125783e-16


In [22]:
Solver__ = Gauss()
Solver__.random_fill_with_zeros(50)
Solver__.solve_()

X__ = Solver__.get_roots()
# X__

In [23]:
Solver___ = Gauss()
Solver___.random_fill_with_zeros(50)
Solver___.solve()

X___ = Solver___.get_roots()
# X__

In [24]:
print(Solver__.epsilon())
print(Solver___.epsilon())

1.0452043956402235e-13
3.2828820242375555e-15
